# Evaluate on AMOS Dataset

## 1. Download data

```bash
# download data (24 GB)
wget https://zenodo.org/records/7262581/files/amos22.zip?download=1 -O amos22.zip
UNZIP_DISABLE_ZIPBOMB_DETECTION=TRUE unzip amos22.zip
cd amos22

# split into MRI and CT (MRI entry ids start with 500)
########################  MRI  ################################

mkdir mri; cd mri
mkdir image_links label_links
cd image_links
for f in $(ls ../../images*/* | grep -E '(Tr|Va)/amos_0[56]')
do
    ln -s $f
done

cd ../label_links
for f in $(ls  ../../labels*/* | grep -E '(Tr|Va)/amos_0[56]')
do
    ln -s $f
done
cd ../..

########################  CT  #################################

mkdir ct; cd ct
mkdir image_links label_links
cd image_links
for f in $(ls  ../../images*/* | grep -E '(Tr|Va)/amos_0[01234]')
do
    ln -s $f
done

cd ../label_links
for f in $(ls  ../../labels*/* | grep -E '(Tr|Va)/amos_0[01234]')
do
    ln -s $f
done
```

## 2. Start Segmentation
I recommend to start with the MRI because the CT dataset is large and segmentation can be time and memory intensive.

```bash
# Run Inference
mrsegmentator -i amos22/mri/image_links --outdir amos22/mri/preds --batchsize 16
mrsegmentator -i amos22/ct/image_links --outdir amos22/ct/preds --split_level 1
```

## 3. Evaluate
For faster execution change the spacing to (5.0, 5.0, 5.0)

In [1]:
import pandas as pd
import eval_data
import mappings
from os.path import join

data_path = "/home/user/amos22"

eval_data.main(
    label_dir=join(data_path, "mri", "label_links"),
    seg_dir=join(data_path, "mri", "preds"),
    dataset_name="csvs/AMOS-MR",
    class_mapping=mappings.amos_to_mrseg,
    spacing=(1.5, 1.5, 3.0),
)

eval_data.main(
    label_dir=join(data_path, "ct", "label_links"),
    seg_dir=join(data_path, "ct", "preds"),
    dataset_name="csvs/AMOS-CT",
    class_mapping=mappings.amos_to_mrseg,
    spacing=(1.5, 1.5, 3.0),
)

data = pd.DataFrame(index=mappings.channels)
data["MRI"] = pd.read_csv("csvs/AMOS-MRI_dsc.csv").mean()
data["CT"] = pd.read_csv("csvs/AMOS-CT_dsc.csv").mean()

pd.options.display.float_format = "{:,.2f}".format
print(data.mean())
data.dropna()

MRI   0.79
CT    0.84
dtype: float64


,MRI,CT
spleen,0.94,0.95
right_kidney,0.95,0.94
left_kidney,0.94,0.94
gallbladder,0.73,0.80
liver,0.96,0.96
stomach,0.87,0.89
pancreas,0.79,0.79
right_adrenal_gland,0.54,0.69
left_adrenal_gland,0.53,0.69
aorta,0.89,0.93
